In [1]:
setwd("../../")
pacman::p_load(ggplot2, dplyr, stringr, lme4, tidyverse, ComplexHeatmap, WGCNA)
options(stringsAsFactors = FALSE)

---
# write eigengenes supplimental table

In [2]:
wgcna_result = readRDS("output/correlation/table/WGCNA_geneTrees.rds")
module_decode = read.csv("output/correlation/table/Module_decode.csv", header = T, row.names = 1)
module_decode$category = str_split_fixed(rownames(module_decode),"\\.",2)[,1]
module_decode$color = str_split_fixed(rownames(module_decode),"\\.",2)[,2]

In [3]:
abun_me = wgcna_result$abun_result$MEs
bioc_me = wgcna_result$bioc_result$MEs
immune_me = wgcna_result$immune_result$MEs
kegg_me = wgcna_result$kegg_result$MEs
abun_me = abun_me[,colnames(abun_me) %in% rownames(module_decode)]
bioc_me = bioc_me[,colnames(bioc_me) %in% rownames(module_decode)]
immune_me = immune_me[,colnames(immune_me) %in% rownames(module_decode)]
kegg_me = kegg_me[,colnames(kegg_me) %in% rownames(module_decode)]

In [4]:
eigengenes_table = list(abun_me, kegg_me, immune_me, bioc_me) %>%
  map(~ .x %>% rownames_to_column("rowname")) %>%
  reduce(full_join, by = "rowname") %>%
  column_to_rownames("rowname")
eigengenes_table_full = rbind(module_decode[,c('decode','member_counts')] %>% t(), eigengenes_table)
write.csv(eigengenes_table_full, "output/correlation/table/Module_Eigengene_Full.csv")

In [5]:
module_decode$Decode = module_decode$decode
module_decode$Decode = str_replace_all(module_decode$Decode, " ", "")
module_decode$Decode = str_replace_all(module_decode$Decode, "\\+", "")
module_decode$Decode = str_replace_all(module_decode$Decode, "\\/", "")
colnames(abun_me) = module_decode[colnames(abun_me),'Decode']
colnames(bioc_me) = module_decode[colnames(bioc_me),'Decode']
colnames(immune_me) = module_decode[colnames(immune_me),'Decode']
colnames(kegg_me) = module_decode[colnames(kegg_me),'Decode']
colnames(eigengenes_table) = module_decode[colnames(eigengenes_table),'Decode']
sample_list = rownames(eigengenes_table)
rownames(module_decode) = module_decode$Decode
module_decode=module_decode[colnames(eigengenes_table),]

In [6]:
meta_data = read.csv("data/metadata/Metadata_061523.csv", row.names = 1, header = T)
meta_data = meta_data[sample_list, ]

In [7]:
score = read.csv("codes/AI/input/score.csv", row.names = 1, header = T)
score = score[sample_list, ]

---
# linear regression

In [8]:
fit_lmm_eigengene <- function(data) {
    variables = colnames(data)
    feature_names = colnames(eigengenes_table)
    data <- cbind(eigengenes_table, data, sample_id_tp1 = meta_data[['sample_id_tp1']])
    
    result_table <- data.frame()
    for (i in feature_names) {
        formula <- as.formula(paste(i, " ~ ", paste(variables, collapse = "+"), "+ (1|sample_id_tp1)"))
        model <- suppressMessages(suppressWarnings(lmer(formula, data = data)))
        summary_mat <- summary(model)$coefficients
        tvalue <- summary_mat[,'t value']
    
    # If it's the first feature, initialize the result_table with row names
    if (dim(result_table)[1] == 0) {
      result_table <- matrix(0, ncol = length(feature_names), nrow = length(tvalue))
      rownames(result_table) <- names(tvalue)
      colnames(result_table) <- feature_names
    }
    result_table[,i] <- tvalue
  }
  return(result_table)
}

In [9]:
data = meta_data[,c('study_ptorhc','illness_duration',
              'age','gender','bmi','race','ethnic',
              'diet_meat','diet_sugar','diet_veg','diet_grains','diet_fruit',
               'antifungals','antibiotics','probiotics','antivirals')]
meta_eigengenes =  fit_lmm_eigengene(data)

In [10]:
score_eigengenes =  fit_lmm_eigengene(score)

---
# Coefficients Heatmap

In [11]:
pdf("output/correlation/figure/Linear_Model_Coefficients.pdf", width = 10, height = 6.5)

col_fun = circlize::colorRamp2(c(-4.5,0,4.5), c("blue","white","red"))
annotation_text = module_decode$category
annotation_text = factor(annotation_text, levels = (unique(annotation_text)))
annotation_colors = module_decode$color
names(annotation_colors) = module_decode$Decode
module_colors = list(WGCNA_Color = annotation_colors)
col_an = HeatmapAnnotation(WGCNA_Color = module_decode$Decode, col = module_colors, 
                           annotation_name_gp = gpar(fontsize = 12),
                           show_legend = FALSE)

meta_plot = Heatmap(meta_eigengenes, cluster_rows = FALSE, cluster_columns = FALSE, 
                    column_split = annotation_text,col = col_fun,column_title=NULL,
                    top_annotation = col_an,
                    row_names_gp = gpar(fontsize = 10),show_heatmap_legend=F)
score_plot = Heatmap(score_eigengenes, cluster_rows = FALSE, cluster_columns = FALSE, col = col_fun,
                    column_split = annotation_text, 
                    column_labels = module_decode$decode,
                    column_names_gp = gpar(fontsize = 12),
                    row_names_gp = gpar(fontsize = 10),  
                    heatmap_legend_param = list(title = "Linear Model \nCoefficients", 
                                                 title_gp = gpar(fontsize = 10)))
meta_plot %v% score_plot
dev.off()

png 
  2

# WGCNA Trees

In [12]:
table(module_decode$category)


    Immune       KEGG Metabolome     Specie 
         9          6          9          7 

In [13]:
pdf("output/correlation/figure/Species_tree.pdf", width = 7/2, height = 2.8)
plotDendroAndColors(wgcna_result$abun_result$geneTree, wgcna_result$abun_result$mergedColors, 
                    main=NULL, dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05, )
dev.off()
pdf("output/correlation/figure/Immune_tree.pdf", width = 9/2, height = 2.65)
plotDendroAndColors(wgcna_result$immune_result$geneTree, wgcna_result$immune_result$mergedColors, 
                    main=NULL, dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05, )
dev.off()
pdf("output/correlation/figure/Metabolome_tree.pdf", width = 9/2, height = 2.65)
plotDendroAndColors(wgcna_result$bioc_result$geneTree, wgcna_result$bioc_result$mergedColors, 
                    main=NULL, dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05, )
dev.off()
pdf("output/correlation/figure/KEGG_tree.pdf", width = 6/2, height = 2.5)
plotDendroAndColors(wgcna_result$kegg_result$geneTree, wgcna_result$kegg_result$mergedColors, 
                    main=NULL, dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05, )
dev.off()

png 
  2

png 
  2

png 
  2

png 
  2